In [100]:
import pandas as pd
import geopandas as gpd

from shared_utils import rt_utils, catalog_utils, rt_dates

from calitp_data_analysis import get_fs, geography_utils
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling, corridor_analysis
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT
import numpy as np
import datetime as dt

ADHOC_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/adhoc/"

# Transit Priority on SHS Feb 2025

## Methodology and Assumptions

* manually create lists of corridors and hotspots based on speedmaps, judgement
* find relevant trip data (segments) for each corridor, from the first segment that intersects the defined corridor to the last.
* only keep routes that travel along the corridor for at least half of the corridor length (exclude cross-street routes)
* exclude trips with apparent zero seconds in corridor, speeds above 80mph, speeds below the 5th or above the 95th percentile.
* calculate speed and travel times for trips through corridor
* for each trip, roughly estimate effects of transit priority
    * corridors: either all trip speeds increase to 16mph if current conditions median speed less than 13.6mph, otherwise all trip speeds increase by 15%
    * hotspots: all trips save 30 seconds of travel time through hotspot
* track total daily trips and peak frequency across all routes
* estimate average trip delay by dividing total minutes of delay by daily trip count
* estimate rider delay for certain examples by multiplying average trip delay by ridership
* "minutes_per_mile" is a metric created by dividing total daily minutes of delay by the length of the corridor, it's intended to provide a rough measure of delay intensity or cost-effectiveness

In [47]:
import importlib
importlib.reload(corridor_analysis)

<module 'segment_speed_utils.corridor_analysis' from '/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py'>

In [3]:
analysis_date = rt_dates.DATES['dec2024']

## need trip-level (pre-aggregation) gdf to properly calculate metrics

In [4]:
segment_speeds = corridor_analysis.import_speedmap_segment_speeds(analysis_date)

In [5]:
frequencies = corridor_analysis.get_max_frequencies(segment_speeds)

/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_cols + ['direction_id']).sum().groupby(


In [6]:
trip_speeds = corridor_analysis.import_trip_speeds(analysis_date)

2.2 percent of segments have no speed


## corridor specification

In [ ]:
# rt_utils.show_full_df(pd.read_parquet('../ca_transit_speed_maps/_rt_progress_2024-12-11.parquet').sort_values(['caltrans_district', 'organization_name']))

## Corridor Measurements

Previous logic:

For each trip, get from the last stop before entering corridor to the first stop after exiting corridor. This was done on stop_sequence

Now,

* first sjoin with aggregated data (has geom). Sjoining on segments is equivalent to previous methodology, since it will yield the last stop before entry to the first stop after exiting...
* avoid doing scheduled delay metric for now...
* 

In [ ]:
corr = pd.read_excel(f'{ADHOC_PATH}corr_hs.xlsx', sheet_name='Corridor Input')
hs = pd.read_excel(f'{ADHOC_PATH}corr_hs.xlsx', sheet_name='Hotspot Input')
corr = corr.astype({'start_segment_id': str, 'end_segment_id': str, 'shape_id':str})
hs = hs.astype({'start_segment_id': str, 'end_segment_id': str, 'shape_id':str})

In [ ]:
row = corr.loc[15]

In [ ]:
gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=segment_speeds,
                                               organization_source_record_id=row.organization_source_record_id,
                                               shape_id=row.shape_id,
                                               start_seg_id=row.start_segment_id,
                                               end_seg_id=row.end_segment_id)

In [ ]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, gdf, trip_speeds)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

In [ ]:
corridor_analysis.validate_corridor_routes(gdf, corridor_trips)

In [ ]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

In [ ]:
corridor_improvements.head(3)

In [ ]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies)

In [ ]:
summarized

In [ ]:
# corr_gdf = corridor_from_sheet(corr, intervention_dict={'trip_mph_floor': 16, 'trip_percent_speedup': 15}, fwy_xpwy_floor = 35)
corr_gdf = corridor_analysis.corridor_from_sheet(corr,
                                                 segment_speeds,
                                                 trip_speeds,
                                                 frequencies,
                                                 intervention_dict={'trip_mph_floor': 16, 'trip_percent_speedup': 15},
                                                 fwy_xpwy_floor = 35)

In [ ]:
corr_gdf.to_excel(f'{ADHOC_PATH}corridors.xlsx')

In [ ]:
# corr_gdf.iloc[1:,:].explore(column='minutes_per_mile')

In [ ]:
hs_gdf = corridor_analysis.corridor_from_sheet(hs,
                             segment_speeds,
                             trip_speeds,
                             frequencies,
                             intervention_dict={'trip_seconds_saved': 30})

In [ ]:
hs_gdf.to_excel(f'{ADHOC_PATH}hotspots.xlsx')

## Discussion

* Which other metrics?

after screening:

ridership/person-hours of delay
accessibility, equity

* Is our list complete?

Include Tempo, Van Ness, can we compare?

* Exclude routes where necessary (current corridor join is just spatial...)

* Add location-specific interventions, and [guidance](https://caltrans.sharepoint.com/:w:/s/DOTPMPHQ-DataandDigitalServices/EdG0YNQcQMBJmKncAuNva9wBjpxVq2sD8p3C5azumXFNRA?e=TO7CbB)

* How much should we focus on freeways?

Yes, include (Bay Bridge, others, SDMTS freeway service, launch service with new HOT...)

District transit plans, express/rapid on freeways...?

## webmaps

In [ ]:
import branca

In [ ]:
import sys
sys.path.append('../ca_transit_speed_maps/')

In [ ]:
import export_legends

In [ ]:
# colors = branca.colormap.step.Reds_08.colors[3:]
# cmap = branca.colormap.LinearColormap(colors=colors).to_step(data=corr_gdf.iloc[1:,:].delay_minutes, method='quant', n=5)

In [ ]:
# corr_gdf.iloc[1:,:].explore(cmap=cmap, column='delay_minutes')

In [ ]:
# labels = [int(x) for x in cmap.tick_labels[1:-1]]
# export_legends.export_legend(cmap, filename='transit_delay_quantiles.svg', inner_labels=labels)

In [ ]:
# colors = branca.colormap.step.Reds_08.colors[3:]
# cmap = branca.colormap.LinearColormap(colors=colors).to_step(data=hs_gdf.delay_minutes, method='quant', n=5)
# hs_gdf.explore(cmap=cmap, column='delay_minutes')

In [ ]:
# labels = [int(x) for x in cmap.tick_labels[1:-1]]
# export_legends.export_legend(cmap, filename='transit_delay_quantiles_hs.svg', inner_labels=labels)

In [ ]:
shn = gpd.read_parquet(rt_utils.SHN_PATH)

In [ ]:
drop_van_ness = corr_gdf.iloc[1:,:]

In [ ]:
drop_van_ness.columns

In [ ]:
gdf = drop_van_ness.drop(columns=['corridor_id', 'intervention_assumption']).rename(columns={'trips_per_hr_peak_directional': 'trips/hr'})

In [ ]:
export_result = rt_utils.set_state_export(shn, subfolder = 'transit_delay/', filename = 'shn',
                    map_type = 'state_highway_network')
shn_state = export_result['state_dict']

In [ ]:
export_result = rt_utils.set_state_export(
    gdf, subfolder = 'transit_delay/', filename=f'corridors_{analysis_date}',
    color_col='delay_minutes', cmap=cmap, legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_delay_quantiles.svg",
    map_title='Corridor Transit Delay Dec 2024 (total daily vehicle-minutes)',
    existing_state = shn_state)

In [ ]:
export_result

In [ ]:
gdf2 = hs_gdf.drop(columns=['corridor_id', 'intervention_assumption']).rename(columns={'trips_per_hr_peak_directional': 'trips/hr'})

In [ ]:
export_result = rt_utils.set_state_export(shn, subfolder = 'transit_delay/', filename = 'shn',
                    map_type = 'state_highway_network')
shn_state = export_result['state_dict']

In [ ]:
export_result = rt_utils.set_state_export(
    gdf2, subfolder = 'transit_delay/', filename=f'hotspots2_{analysis_date}',
    color_col='delay_minutes', cmap=cmap, legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_delay_quantiles_hs.svg",
    map_title='Hotspot Transit Delay Dec 2024 (total daily vehicle-minutes)',
    existing_state = shn_state)

In [ ]:
export_result

## bespoke van ness

* v2 warehouse doesn't include RT before Sep 2022, and Van Ness opened Apr 2022
* get old rt_analysis code working

In [ ]:
old_sfmta = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/282_2022-02-08.parquet')

In [ ]:
import rt_analysis

In [ ]:
rt_analysis.rt_filter_map_plot.from_gcs?

In [ ]:
from rt_analysis import rt_filter_map_plot

In [ ]:
importlib.reload(rt_filter_map_plot)

In [ ]:
sfmta = rt_analysis.rt_filter_map_plot.from_gcs(282, '2022-02-08')

In [ ]:
sfmta.set_filter(route_names=['49'])

In [ ]:
sfmta.segment_speed_map?

In [ ]:
sfmta.segment_speed_map()

In [ ]:
sfmta.autocorridor?

In [ ]:
sfmta.autocorridor('194252', [27, 37])

In [ ]:
old_van_ness = sfmta.corridor_metrics()

In [ ]:
new_van_ness = corr_gdf.query('corridor_name == "US101 Van Ness"')

In [ ]:
old_van_ness

In [ ]:
sfmta.segment_speed_map(corridor=True, no_render=True)

In [ ]:
old_segments = sfmta.stop_segment_speed_view[sfmta.stop_segment_speed_view.corridor]

In [ ]:
new_van_ness

In [ ]:
row = corr.iloc[3,:]

In [ ]:
row

In [ ]:
gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=segment_speeds,
                                               organization_source_record_id=row.organization_source_record_id,
                                               shape_id=row.shape_id,
                                               start_seg_id=row.start_segment_id,
                                               end_seg_id=row.end_segment_id)

In [ ]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, gdf, trip_speeds)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

In [ ]:
corridor_results.corridor_speed_mph.median()

In [ ]:
(8.4 - 6.8) / 6.8

In [ ]:
corridor_trips = corridor_trips.query('speed_mph <= 30 & speed_mph > speed_mph.quantile(.05)')

In [ ]:
corridor_trips = corridor_trips.query('route_short_name == "49"')

In [ ]:
corridor_trips.columns

In [ ]:
old_segments = old_segments.query('speed_mph <= 30')

In [ ]:
import matplotlib.pyplot as plt

plt.hist(old_segments['speed_mph'],  
         alpha=0.5, # the transaparency parameter 
         label='Feb 2022',
         bins=20) 
  
plt.hist(corridor_trips['speed_mph'], 
         alpha=0.5, 
         label='Feb 2023',
         bins=20) 
  
plt.legend(loc='upper right') 
plt.title('Van Ness BRT segment speeds')
plt.xlabel('Speed (mph)')
plt.ylabel('Trips (count)')
plt.show()

## Extra LA Metro Corridors

### SR2 Glendale Bl Alvarado St

* multi-route corridor

In [7]:
f'{ADHOC_PATH}sr2_glendale_alvarado.geojson'

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/adhoc/sr2_glendale_alvarado.geojson'

In [12]:
from calitp_data_analysis import get_fs

In [13]:
fs = get_fs()

In [33]:
with fs.open(f'{ADHOC_PATH}sr2_glendale_alvarado.geojson') as f:
    sr2_glen_alv = gpd.read_file(f).to_crs(geography_utils.CA_NAD83Albers_m)
    sr2_glen_alv['corridor_distance_meters']= sr2_glen_alv.geometry.iloc[0].length
    sr2_glen_alv.geometry = sr2_glen_alv.buffer(corridor_analysis.CORRIDOR_BUFFER)

In [144]:
# sr2_glen_alv.explore()

In [74]:
sr2_glen_alv['corridor_id'] = 'sr2_glendale_alvarado'

In [75]:
sr2_glen_alv

,geometry,corridor_distance_meters,corridor_id,corridor_name
0,"POLYGON ((160694.254 -434767.547, 160656.841 -...",2253.076351,sr2_glendale_alvarado,sr2_glendale_alvarado


In [77]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, sr2_glen_alv, trip_speeds,
                                                     custom_relevance_meters = 200)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

0.0 percent of trips with zero seconds
0.0 percent of trips with speeds > 80mph
10.1 percent of trips with speed below 5th or above 95%ile


In [97]:
# corridor_analysis.validate_corridor_routes(sr2_glen_alv, corridor_trips)

In [79]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

median: 9.37mph... mph floor mode: 16mph


In [80]:
corridor_improvements.head(3)

,trip_instance_key,corridor_meters,corridor_seconds,corridor_speed_mps,corridor_speed_mph,route_short_name,route_id,shape_array_key,shape_id,schedule_gtfs_dataset_key,time_of_day,corridor_id,improved_corridor_speed_mph,intervention_assumption,improved_corridor_speed_mps,improved_corridor_seconds
0,00f14b5ff4aca7b5ada9bed374e32b31,1219.700581,211.0,5.780571,12.931138,2,2-13183,d592976a971b9ef47ec9bd645e34c04f,21124_JUNE24,0666caf3ec1ecc96b74f4477ee4bc939,Midday,sr2_glendale_alvarado,16.0,trips achieve 16mph or existing spd if higher,7.152436,170.529388
1,0146eb25c814fad79928485225b8b5db,1219.700581,262.0,4.655346,10.414008,2,2-13183,d592976a971b9ef47ec9bd645e34c04f,21124_JUNE24,0666caf3ec1ecc96b74f4477ee4bc939,Early AM,sr2_glendale_alvarado,16.0,trips achieve 16mph or existing spd if higher,7.152436,170.529388
2,01565dd825d4848418cca4439eda7a0d,2110.918964,378.0,5.584442,12.492396,603,603-13183,387f7d042aa398a8f2b374757bed7078,6030026_JUNE24,0666caf3ec1ecc96b74f4477ee4bc939,Early AM,sr2_glendale_alvarado,16.0,trips achieve 16mph or existing spd if higher,7.152436,295.132858


In [81]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies)

In [93]:
summarized

,corridor_id,schedule_gtfs_dataset_key,intervention_assumption,corridor_seconds,improved_corridor_seconds,delay_seconds,delay_minutes,route_short_name,route_id,median_corridor_mph,n_trips_daily,trips_hr_sch,corridor_name,corridor_notes,district
0,sr2_glendale_alvarado,0666caf3ec1ecc96b74f4477ee4bc939,trips achieve 16mph or existing spd if higher,156886.0,88973.1,67912.9,1131.9,"[2, 603, 92]","[2-13183, 603-13183, 92-13183]",9.4,427,"[10.6, 5.3, 7.6]",sr2_glendale_alvarado,,7


In [72]:
corridor_improvements.corridor_meters.iloc[0] / rt_utils.METERS_PER_MILE

0.7578886882167059

In [94]:
summarized.delay_minutes / .75

0    1509.2
Name: delay_minutes, dtype: float64

In [95]:
summarized.trips_hr_sch.map(lambda x: sum(x))

0    23.5
Name: trips_hr_sch, dtype: float64

In [96]:
summarized.delay_minutes / 427

0    2.65082
Name: delay_minutes, dtype: float64

### SR2 Santa Monica Bl (Westside I405-Ohio)

* planned peak-hour lanes, assess difference with all-day
* 7a-9a, 4p-6p

In [188]:
importlib.reload(corridor_analysis)

<module 'segment_speed_utils.corridor_analysis' from '/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py'>

#### midday

In [225]:
analysis_date

'2024-12-11'

In [226]:
start_mid = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('09:00'))
end_mid = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('16:00'))

In [227]:
gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=segment_speeds,
                                               organization_source_record_id='recPnGkwdpnr8jmHB',
                                               shape_id='40364_JUNE24',
                                               start_seg_id='5979-6041-2',
                                               end_seg_id='5991-14424-1')

In [228]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, gdf, trip_speeds,
                                                     start_datetime=start_mid,
                                                     end_datetime=end_mid)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

0.0 percent of trips with zero seconds
0.0 percent of trips with speeds > 80mph
11.3 percent of trips with speed below 5th or above 95%ile


In [229]:
# corridor_analysis.validate_corridor_routes(gdf, corridor_trips) #  metro 4, bbb 1 looks good

In [230]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

median: 9.15mph... mph floor mode: 16mph


In [231]:
# corridor_improvements.head(3)

In [232]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies, gdf)

In [233]:
corridor_analysis.combine_corridor_operators(summarized)

,corridor_id,intervention_assumption,corridor_miles,delay_minutes,minutes_per_mile,median_corridor_mph,trips_per_hr_peak_directional,n_trips_daily,average_trip_delay
0,2365467368501946497,trips achieve 16mph or existing spd if higher,1.4,730.6,505.4,9.4,27.4,125,5.8


#### eve

In [198]:
analysis_date

'2024-12-11'

In [199]:
start_eve = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('18:00'))
end_eve = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('23:59'))

In [200]:
gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=segment_speeds,
                                               organization_source_record_id='recPnGkwdpnr8jmHB',
                                               shape_id='40364_JUNE24',
                                               start_seg_id='5979-6041-2',
                                               end_seg_id='5991-14424-1')

In [201]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, gdf, trip_speeds,
                                                     start_datetime=start_eve,
                                                     end_datetime=end_eve)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

1.1 percent of trips with zero seconds
0.0 percent of trips with speeds > 80mph
11.2 percent of trips with speed below 5th or above 95%ile


In [202]:
# corridor_analysis.validate_corridor_routes(gdf, corridor_trips) #  metro 4, bbb 1 looks good

In [203]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

median: 12.01mph... mph floor mode: 16mph


In [204]:
# corridor_improvements.head(3)

In [205]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies, gdf)

In [206]:
corridor_analysis.combine_corridor_operators(summarized)

,corridor_id,intervention_assumption,corridor_miles,delay_minutes,minutes_per_mile,median_corridor_mph,trips_per_hr_peak_directional,n_trips_daily,average_trip_delay
0,2365467368501946497,trips achieve 16mph or existing spd if higher,1.4,241.3,167.0,11.7,27.4,79,3.1


#### am

In [234]:
analysis_date

'2024-12-11'

In [235]:
start_am = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('07:00'))
end_am = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('09:00'))

In [236]:
gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=segment_speeds,
                                               organization_source_record_id='recPnGkwdpnr8jmHB',
                                               shape_id='40364_JUNE24',
                                               start_seg_id='5979-6041-2',
                                               end_seg_id='5991-14424-1')

In [237]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, gdf, trip_speeds,
                                                     start_datetime=start_am,
                                                     end_datetime=end_am)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

0.0 percent of trips with zero seconds
0.0 percent of trips with speeds > 80mph
10.3 percent of trips with speed below 5th or above 95%ile


In [238]:
# corridor_analysis.validate_corridor_routes(gdf, corridor_trips) #  metro 4, bbb 1 looks good

In [239]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

median: 10.1mph... mph floor mode: 16mph


In [240]:
# corridor_improvements.head(3)

In [241]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies, gdf)

In [242]:
am = corridor_analysis.combine_corridor_operators(summarized)

In [243]:
am

,corridor_id,intervention_assumption,corridor_miles,delay_minutes,minutes_per_mile,median_corridor_mph,trips_per_hr_peak_directional,n_trips_daily,average_trip_delay
0,2365467368501946497,trips achieve 16mph or existing spd if higher,1.4,138.2,95.6,10.2,27.4,35,3.9


#### pm

In [244]:
analysis_date

'2024-12-11'

In [245]:
start_pm = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('16:00'))
end_pm = dt.datetime.combine(dt.date.fromisoformat(analysis_date), dt.time.fromisoformat('18:00'))

In [246]:
gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=segment_speeds,
                                               organization_source_record_id='recPnGkwdpnr8jmHB',
                                               shape_id='40364_JUNE24',
                                               start_seg_id='5979-6041-2',
                                               end_seg_id='5991-14424-1')

In [247]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, gdf, trip_speeds,
                                                     start_datetime=start_pm,
                                                     end_datetime=end_pm)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

0.0 percent of trips with zero seconds
0.0 percent of trips with speeds > 80mph
14.3 percent of trips with speed below 5th or above 95%ile


In [248]:
# corridor_analysis.validate_corridor_routes(gdf, corridor_trips) #  metro 4, bbb 1 looks good

In [249]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

median: 5.2mph... mph floor mode: 16mph


In [250]:
# corridor_improvements.head(3)

In [251]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies, gdf)

In [252]:
pm = corridor_analysis.combine_corridor_operators(summarized)

In [253]:
pm

,corridor_id,intervention_assumption,corridor_miles,delay_minutes,minutes_per_mile,median_corridor_mph,trips_per_hr_peak_directional,n_trips_daily,average_trip_delay
0,2365467368501946497,trips achieve 16mph or existing spd if higher,1.4,511.5,353.8,5.6,27.4,36,14.2
